In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd '/content/drive/My Drive/intelligent_chatbot'

/content/drive/My Drive/intelligent_chatbot


In [3]:
ls

chatbot.ipynb  intents.json    training.py
classes.pkl    training.ipynb  words.pkl


In [4]:
import random
import json
import pickle
import numpy as np

In [6]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

#from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', ',', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#print(documents)
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])


random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])


model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)
print('done')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/200
7/7 [==============================] - 1s 2ms/step - loss: 1.7405 - accuracy: 0.2656
Epoch 2/200
7/7 [==============================] - 0s 2ms/step - loss: 1.7669 - accuracy: 0.2016
Epoch 3/200
7/7 [==============================] - 0s 2ms/step - loss: 1.7367 - accuracy: 0.2153
Epoch 4/200
7/7 [==============================] - 0s 2ms/step - loss: 1.5774 - accuracy: 0.4372
Epoch 5/200
7/7 [==============================] - 0s 2ms/step - loss: 1.6095 - accuracy: 0.4236
Epoch 6/200
7/7 [==============================] - 0s 2ms/step - loss: 1.5867 - accuracy: 0.3955
Epoch 7/200
7/7 [==============================] - 0s 2ms/step - loss: 1.5783 - accuracy: 0.4518
Epoch 8/200
7/7 [==============================] - 0s 2ms/step - loss: 1.5722 - accuracy: 0.5306
Epoch 9/200
7/7 [==============================] - 0s 2ms/step - loss: 1.4198 - accuracy: 0.5202
Epoch 10/200
7/7 [==============================] - 0s 2ms/step - loss: 1.2681 - accuracy: 0.6616
Epoch 11/200
7/7 [===========